In [ ]:
# -*- coding: utf-8 -*-
"""
@author: Rukang Xu
"""

In [ ]:
import sys
print(sys.version)

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Notes for package installation:
> When specifying `-e` or `develop`, MMDetection is installed on **dev** mode , any local modifications made to the code will take effect without reinstallation.

In [ ]:
# install pytorch with specified version based on cuda110 and available mmcv-full source for cuda110 and pytorch1.7.0
!pip install '../input/cellis/mmdetection_cuda110/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '../input/cellis/mmdetection_cuda110/torchvision-0.8.0-cp37-cp37m-linux_x86_64.whl' --no-deps
import torch, torchvision
torch_version = torch.__version__
print(torch.__version__, torch.cuda.is_available())


In [ ]:
# check the gpu device being used
if torch.cuda.is_available():
  print('the device name is:')
  print(torch.cuda.get_device_name(torch.cuda.current_device()))
  print('Memory Usage:')
  print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
  print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
  print('No gpu device is available.')

In [ ]:

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.18 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html

# Install mmdetection
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
#!rm -rf CBNetV2
#!git clone https://github.com/VDIGPKU/CBNetV2.git
#%cd CBNetV2
#!pip install -e .
!git clone https://github.com/SwinTransformer/Swin-Transformer-Object-Detection.git 
%cd Swin-Transformer-Object-Detection


!pip install -e .

# install pandas to deal with current dataset
!pip install pandas

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow

# install opencv
!pip install opencv-python

# install pyyaml
!pip install pyyaml

# install pathlib2, an alternative to os.path
!pip install pathlib2

# install tdam which displays nice progress bars, ipywidgets for tqdm's notebook support
!pip install tqdm
!pip install ipywidgets

# install sklearn
!pip install scikit-learn

In [ ]:
# check TensorFlow installation when it is needed
# import tensorflow as tf
# tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# import other useful python packages
import pathlib2
import os
print(f'the current work dir is: {os.getcwd()}')
import time
import os.path as osp
import PIL
import numpy as np
import pandas as pd
import mmcv
import matplotlib.pyplot as plt
import cv2

## Macros

In [ ]:
from easydict import EasyDict as edict

__C = edict()
local_cfg = __C


__C.USE_GOOGLE_DRIVE = False
__C.CURRENT_DIR = pathlib2.Path.cwd()
print("current dir: ", __C.CURRENT_DIR.as_posix())
# the highest level dir that holds all relevant files
__C.PROJECT_DIR = __C.CURRENT_DIR.parent
print("project dir: ", __C.PROJECT_DIR.as_posix())
__C.DATA_DIR = __C.PROJECT_DIR.parent / 'cellisdata'
print("useful dataset dir: ", __C.PROJECT_DIR.as_posix())

__C.SPLIT_IN_THREE = True
__C.TEST_SIZE = 0.3
__C.VAL_SIZE = 1/float(6)
__C.RANDOM_STATE = 42
__C.FEATURES = ['id', 'annotation', 'width', 'height', 'cell_type']

__C.HEIGHT, __C.WIDTH = 520, 704

#### Check the dataset

In [ ]:
# Check the working directory

# Install tree first
# TODO this following origin file may be not general to each case
!apt-get -q install tree
!tree /kaggle/working/ -L 1

In [ ]:
!cp -r /kaggle/input/apexforcellis/apex /kaggle/working/
!ls /kaggle/working/apex

In [ ]:
%cd /kaggle/working/apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

%cd /kaggle/working/Swin-Transformer-Object-Detection
import os 
os.getcwd()

In [ ]:
!mkdir -p /kaggle/working/Swin-Transformer-Object-Detection/checkpoints/

**Resume from the checkpoint if needed**

In [ ]:
'''
!cp "$ds_path_pre"/epoch_3.pth "$local_cfg.PROJECT_DIR"/CBNetV2
from mmcv import Config
cfg = Config.fromfile('configs/cbnet/cascade_mask_rcnn_cbv2_swin_small_patch4_window7_mstrain_400-1400_adamw_2x_live_2.py')
cfg.keys()
cfg.resume_from = './epoch_3.pth'
'''

#### Create */kaggle/temp* for increasing the available usable disk space

In [ ]:
# build the */kaggle/temp/*
!mkdir -p /kaggle/temp/

### Check the checkpoints and log infos and download them

In [ ]:
# check the contents mainly in terms of the checkpoints and log info
!ls /kaggle/temp/

## Competition dataset

### Copy the relevant files for training

In [ ]:
!mkdir /kaggle/working/data/
!mkdir /kaggle/working/data/livecell_images/
!mkdir /kaggle/working/data/livecell_images/livecell_images/

In [ ]:
!ls /kaggle/input/competconfig/

In [ ]:
!ls /kaggle/working/data/livecell_images/livecell_images/

In [ ]:
#copy config and images and annots 
!cp /kaggle/input/competconfig/mask_rcnn_r50_fpn_compet.py /kaggle/working/Swin-Transformer-Object-Detection/configs/mask_rcnn/
!cp -RT /kaggle/input/competdata/train /kaggle/working/data/livecell_images/livecell_images
!cp -RT /kaggle/input/competdataplus/ /kaggle/working/data/livecell_images/
#check the contents of dst dir
!ls /kaggle/working/data/livecell_images/

In [ ]:
!cp -RT /kaggle/input/competdata/train /kaggle/working/data/livecell_images/train

In [ ]:
#copy the pretrained checkpoint from the model trained on livecell
!cp /kaggle/input/cellisckpt/epoch_11.pth /kaggle/working/Swin-Transformer-Object-Detection/checkpoints/

**Check the validity of annotation files w.r.t competition**

In [ ]:
val_ann_file = '/kaggle/working/data/livecell_images/competition_coco_val_v2.json'
file_format = str(val_ann_file).split('.')[-1]
val_data_infos = mmcv.load(str(val_ann_file), file_format=file_format)
val_data_infos.keys()

### Start training

In [ ]:
'''
%cd /kaggle/working/Swin-Transformer-Object-Detection
cfg = mmcv.Config.fromfile('configs/mask_rcnn/mask_rcnn_r50_fpn_compet.py')
cfg.resume_from = '/kaggle/temp/epoch_10.pth'
cfg.dump('configs/mask_rcnn/mask_rcnn_r50_fpn_compet.py')
'''

In [ ]:
%cd /kaggle/working/Swin-Transformer-Object-Detection
print(os.getcwd())
########################################################
#cfg = mmcv.Config.fromfile('configs/cbnet/cascade_mask_rcnn_cbv2_swin_tiny_patch4_window7_mstrain_480-800_adamw_2x_compet.py')
#cfg.load_from = './checkpoints/epoch_2.pth'
########################################################
!tools/dist_train.sh configs/mask_rcnn/mask_rcnn_r50_fpn_compet.py 1